In [12]:
import pointblank as pb
import pandas as pd
import os

In [13]:
df = pd.read_csv("../data/drugs.csv")

In [14]:
print(list(df.columns))
print(len(df.columns))

['STATE', 'STATENAME', 'ST_CASE', 'VEH_NO', 'PER_NO', 'DRUGSPEC', 'DRUGSPECNAME', 'DRUGMETHOD', 'DRUGMETHODNAME', 'DRUGRES', 'DRUGRESNAME', 'DRUGQTY', 'DRUGQTYNAME', 'DRUGACTQTY', 'DRUGACTQTYNAME', 'DRUGUOM', 'DRUGUOMNAME']
17


In [ ]:
expected_schema = pb.Schema(
    columns=[
    ('STATE' , 'int64'),
    ('STATENAME', 'object'),
    ('ST_CASE', 'int64'),
    ('VEH_NO', 'int64'),
    ('PER_NO', 'int64'),
    ('DRUGSPEC', 'int64'),
    ('DRUGSPECNAME', 'object'),
    ('DRUGMETHOD', 'int64'),
    ('DRUGMETHODNAME', 'object'),
    ('DRUGRES', 'int64'),
    ('DRUGRESNAME', 'object'),
    ('DRUGQTY', 'int64'),
    ('DRUGQTYNAME', 'object'),
    ('DRUGACTQTY', 'float64'),
    ('DRUGACTQTYNAME', 'object'),
    ('DRUGUOM', 'int64'),
    ('DRUGUOMNAME', 'object'),
    ('DRUGMETHOD_valid', 'int64')
    ])

# The DRUGMETHOD field is valid if its value is either:
# - In the range 1 to 26 (inclusive), or
# - One of the specific codes: 0, 96, 97, or 99
#
# Applying these two validation rules separately using:
#   .col_vals_between(columns="DRUGMETHOD", left=1, right=26)
#   .col_vals_in_set(columns="DRUGMETHOD", set=[00,96,97,99])
# results in partial failures, since each rule only validates part of the allowed values.
#
# To address this, we combined both conditions into a single boolean column (`DRUGMETHOD_valid`)
# indicating whether each row meets either condition.
#
# Since pointblank does not currently support validation on boolean columns,
# we convert the boolean values to integers (True → 1, False → 0),
# and then validate that all values in the new column equal 1.

df['DRUGMETHOD_valid'] = (df['DRUGMETHOD'].isin([00,96,97,99]) | ((df['DRUGMETHOD'] >=1) & (df['DRUGMETHOD'] <= 26))).astype(int)

validation_report = (
    pb.Validate(
        df, 
        label="Validate Drugs Data Report",
        thresholds=pb.Thresholds(warning=1, error= 20, critical=0.10)
        )
    .col_vals_between(columns="STATE", left=1, right=56)    # Step 1
    .col_vals_regex(columns="STATENAME", pattern=r'^[A-Za-z ]+$')   # Step 2
    .col_vals_between(columns='VEH_NO', left=0, right=99)   # Step 3
    .col_vals_between(columns='PER_NO', left=1, right=99)   # Step 4
    .col_vals_in_set(columns="DRUGSPEC", set=[0,1,2,11,12,13,14,15,96,97,98,99])    # Step 5
    .col_vals_eq(columns='DRUGMETHOD_valid', value=1)   # Step 6
    .col_vals_in_set(columns="DRUGQTY", set=[0,1,2,3,4,96,97,98,99])    # Step 7
    .col_vals_in_set(columns='DRUGUOM', set=[1,2,3,4,5,6,7,8,-9])   # Step 8
    .col_vals_not_null(columns=["PER_NO","VEH_NO","STATENAME"]) # Step 9, 10, 11 for 3 columns
    # 17 given column plus 1 additional column
    .col_count_match(18)    # Step 12
    .col_schema_match(schema = expected_schema) # Step 13
    .interrogate()
)

validation_report

In [16]:

# Export Validation report to html
result_folder = "../validation_result"
os.makedirs(result_folder, exist_ok=True)
validation_report.get_tabular_report().write_raw_html(f"{result_folder}/drugs_data_validation_report.html")


In [ ]:
# i refers to the index (position) of the validation step in the pointblank.Validate() pipeline starting from 1
validation_report.get_step_report(i=3)